# Pipeline
##### Laboratorio 4

- Diego Granada 201922383
- Felipe Russi 201914996
- Veronica Escobar 201922197

## Carga de datos

In [8]:
SHOW_PROFILE = True # REVISAR ANTES DE ENTREGAR
!pip install -r requirements.txt

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [9]:
import pandas as pd
pd.set_option('display.max_columns', 25) # Número máximo de columnas a mostrar
pd.set_option('display.max_rows', 50) # Numero máximo de filas a mostar
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score, mean_squared_error
from sklearn.metrics import plot_confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold 
from sklearn import preprocessing
import matplotlib as mplt
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn import tree
import pickle
from pandas_profiling import ProfileReport

pd.set_option('display.max_columns', 25) # Número máximo de columnas a mostrar
pd.set_option('display.max_rows', 50) # Numero máximo de filas a mostar

In [10]:
#creaDataFrame
train_original_data = pd.read_csv('data/university_admission_train.csv', sep=',', encoding = 'utf-8', index_col=0)
test_original_data = pd.read_csv('data/university_admission_test.csv', sep=',', encoding = 'utf-8', index_col=0)
train_original_data.head(5)

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Admission Points
0,479,327,113,4,4.00,2.77,8.88,1,84.47
1,446,301,92,1,1.85,1.50,7.71,0,45.08
2,336,297,100,1,2.41,1.59,7.89,0,47.42
3,20,303,98,3,3.50,3.00,8.50,0,62.00
4,432,320,94,2,1.38,3.50,8.78,1,73.00


## Diccionario de Datos

### Diccionario

|Variable|Descripción|
|---|---|
|Serial No.| Application id. Unique to each applicant |
|GRE Score| The Graduate Record Examination. An exam designed to measure overall academic readiness. 0 to 340 |
|TOEFL Score| Score in the Test of English as a Foreign Language obtained by the applicant. 0 to 120 |
|University Rating| Quality of the university where the applicant graduated. 0 to 5 |
|SOP| Statement of Purpose Strength. 0 to 5 |
|LOR| Letter of recomendation Strength. 0 to 5 |
|CPGA| Cumulative Grade Point Average. An score to measure academic performance in bachelor programs. 0 to 10|
|Research Experience| Does the applicant have research experience?. Either 0 or 1|
|Admission Points| Points obtained by the applicant in the university Masters Programs admission exam. 0 to 150|


## Entendimiento de datos preliminar

In [11]:
print(train_original_data.shape)
train_original_data.info()

(1569, 9)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1569 entries, 0 to 1568
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Serial No.         1569 non-null   int64  
 1   GRE Score          1569 non-null   int64  
 2   TOEFL Score        1569 non-null   int64  
 3   University Rating  1569 non-null   int64  
 4   SOP                1569 non-null   float64
 5   LOR                1569 non-null   float64
 6   CGPA               1569 non-null   float64
 7   Research           1569 non-null   int64  
 8   Admission Points   1504 non-null   float64
dtypes: float64(4), int64(5)
memory usage: 122.6 KB


In [15]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 175 entries, 117 to 483
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   GRE Score          175 non-null    int64  
 1   TOEFL Score        175 non-null    int64  
 2   University Rating  175 non-null    int64  
 3   SOP                175 non-null    float64
 4   LOR                175 non-null    float64
 5   CGPA               175 non-null    float64
 6   Research           175 non-null    int64  
dtypes: float64(3), int64(4)
memory usage: 10.9 KB


In [14]:
train_data = train_original_data.copy()
test_data = test_original_data.copy()

train_data.describe()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Admission Points
count,1569.000000,1569.000000,1569.000000,1569.000000,1569.000000,1569.000000,1569.000000,1569.000000,1504.000000
mean,239.022945,307.669216,101.632250,2.873805,2.980848,3.013907,8.342231,0.469726,66.744328
std,149.802641,17.857559,12.386238,1.245147,1.234802,1.214377,0.812729,0.499242,19.253029
min,1.000000,257.000000,69.000000,1.000000,0.020000,0.000000,6.100000,0.000000,34.000000
25%,94.000000,298.000000,96.000000,2.000000,2.000000,2.050000,7.750000,0.000000,48.000000
50%,236.000000,309.000000,104.000000,3.000000,3.000000,3.000000,8.330000,0.000000,67.000000
75%,377.000000,322.000000,111.000000,4.000000,4.000000,4.000000,9.040000,1.000000,84.000000
max,500.000000,340.000000,120.000000,5.000000,5.000000,5.000000,9.920000,1.000000,145.500000


## Configurando el pipeline

Ahora se configurara el pipeline de la misma forma que se hizo originalmente en el laboratorio 3. Si se quiere ver mas sobre la labor de perfilar los datos y como se exploraron para encontrar que operaciones se incluirían en el pipeline refierase a la carpeta en la entrega que contiene el laboratorio 3 original, donde todo esto se describe a mas detalle. En particular aqui solamente se eliminan los valores nulos y los outliers.

In [28]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures

#Eliminacion de valores nulos
train_data.dropna(inplace=True)
test_data.dropna(inplace=True)

# Delete outliers
train_data = train_data[train_data['Admission Points'] < 90]

# Datos
selected_cols = ["University Rating", "Research", "CGPA"]
X_test_base = train_data[selected_cols]
X_train, X_test, y_train, y_test = train_test_split(X_base, y_base, test_size=0.2, random_state=42, shuffle=True)


# Creacion del pipeline
p3 = Pipeline([('initial',ColumnTransformer([("selector", 'passthrough', selected_cols)])),
               ('imputer', SimpleImputer(missing_values=np.nan, strategy='median')),
               ('model', LinearRegression())])

p3 = p3.fit(X_base,y_base)

p3

Pipeline(steps=[('initial',
                 ColumnTransformer(transformers=[('selector', 'passthrough',
                                                  ['University Rating',
                                                   'Research', 'CGPA'])])),
                ('imputer', SimpleImputer(strategy='median')),
                ('model', LinearRegression())])

In [30]:
print('Score',p3.score(X_base,y_base))
print('MSE:', mean_squared_error(y_base, p3.predict(X_base)))
print('RMSE:', np.sqrt(mean_squared_error(y_base, p3.predict(X_base))))

Score 0.6075201833897046
MSE: 100.92840127928004
RMSE: 10.046312820098727


### Evaluacion del Pipeline

In [25]:
# Score del pipeline
p3.score(X_base,y_base)

0.6075201833897046

## Exportar el pipeline
En esta seccion se exporta el pipeline usando la libreria de joblib. joblib es un set de herramientas que permiten crear pipelines livianas en python. 

In [34]:
import joblib
joblib.dump(p3, 'modelol.joblib')

filename = 'modelol.joblib' # Ubicación del archivo entregado

Para verificar el correcto funcionamiento del pipeline se ejecutara como prueba.

In [35]:
df_recent = pd.read_csv('data/university_admission_test.csv') # Lectura de los datos recientes

# Lee el archivo y carga el modelo
pipeline = joblib.load(filename)

y_predicted =  pipeline.predict(df_recent)

y_predicted

array([63.12308082, 74.65525114, 62.64868481, 84.37146239, 59.30582235,
       52.89790178, 82.59007511, 82.39921218, 60.26013697, 85.26215603,
       50.79840962, 51.87996619, 73.31921068, 57.01546727, 48.76253844,
       83.22628485, 90.33403615, 63.6666204 , 81.50851854, 51.43461936,
       77.0092272 , 86.53457552, 88.29816496, 60.70548379, 56.50649948,
       46.80808607, 90.14317322, 51.18013547, 70.5198878 , 63.63204862,
       56.20619245, 47.57153777, 85.8347448 , 50.16219987, 87.66195522,
       65.70249158, 84.24422044, 48.46223141, 58.41512871, 52.64341788,
       87.15298742, 49.844095  , 53.3432486 , 69.37471026, 56.46067635,
       52.5797969 , 65.70249158, 79.91799418, 45.02669879, 53.53411152,
       51.05289352, 58.72198224, 64.81179794, 92.36990733, 59.70534605,
       74.08266237, 55.18825686, 87.15298742, 74.52800919, 54.87015199,
       73.38283165, 83.73525265, 51.24375644, 54.10670029, 52.32531301,
       60.3125066 , 83.73525265, 81.66480968, 72.06458903, 57.01

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1b26161d-4db3-4dff-ad21-929766523497' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>